In [18]:
!pip install scikit-uplift
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split, GridSearchCV
from sklift.models import SoloModel, ClassTransformation, TwoModels
from sklift.metrics import uplift_at_k, uplift_auc_score


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [20]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/x5_data/prepared_clients_data.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/x5_data/prepared_test_clients_data.csv')

train.head()

,client_id,first_issue_date,first_redeem_date,age,gender,redeem_issue_delta,treatment_flg,purchased,express_points_spent_mean,express_points_spent_sum,regular_points_spent_mean,regular_points_spent_sum,purchase_sum_mean,purchase_sum_sum,alcohol,regular_points_received_last_month,avg_spend_per_day
0,ad6561e2d8,17503.0,17674.0,50.0,F,171.0,1,1,0.000000,0.0,-1.600000,-64.0,270.035250,10801.41,0.025,17.7,95.587699
1,7c1ccbf93f,17480.0,17584.0,24.0,F,104.0,1,1,0.000000,0.0,0.000000,0.0,425.500000,851.00,0.000,9.4,60.785714
2,b58fadcab6,17472.0,17789.0,36.0,U,317.0,1,1,0.000000,0.0,-9.363636,-309.0,429.447576,14171.77,0.000,18.3,124.313772
3,e99e6fabb9,17667.0,17681.0,79.0,F,14.0,0,0,0.000000,0.0,0.000000,0.0,220.923077,2872.00,0.000,4.4,27.883495
4,27fb6f8520,17364.0,17515.0,34.0,F,151.0,1,1,-1.666667,-50.0,-0.133333,-4.0,329.540333,9886.21,0.000,26.7,94.154381


In [21]:
test.head()

,client_id,first_issue_date,first_redeem_date,age,gender,redeem_issue_delta,express_points_spent_mean,express_points_spent_sum,regular_points_spent_mean,regular_points_spent_sum,purchase_sum_mean,purchase_sum_sum,alcohol,regular_points_received_last_month,avg_spend_per_day
0,a9a604ed6e,17787.0,17684.0,36.0,F,-103.0,0.0,0.0,0.000000,0.0,629.506000,3147.53,0.000000,8.5,35.767386
1,ebd7360016,17350.0,17410.0,63.0,F,60.0,0.0,0.0,-1.451613,-45.0,183.244194,5680.57,0.000000,7.2,49.829561
2,908cd9b8e8,17620.0,17725.0,49.0,F,105.0,0.0,0.0,-4.647059,-79.0,364.296471,6193.04,0.000000,6.6,60.716078
3,dceb8ce861,17445.0,17754.0,46.0,U,309.0,0.0,0.0,-135.500000,-813.0,1377.543333,8265.26,0.000000,47.5,150.277455
4,f4f0ac6b06,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-9.500000,-114.0,746.666667,8960.00,0.083333,64.7,94.315789


In [22]:
train = train[['first_redeem_date', 'express_points_spent_mean', 'express_points_spent_sum', 'regular_points_received_last_month', 'avg_spend_per_day', 'treatment_flg', 'purchased']]
test = test[['first_redeem_date', 'express_points_spent_mean', 'express_points_spent_sum', 'regular_points_received_last_month', 'avg_spend_per_day']]

train.head()

,first_redeem_date,express_points_spent_mean,express_points_spent_sum,regular_points_received_last_month,avg_spend_per_day,treatment_flg,purchased
0,17674.0,0.000000,0.0,17.7,95.587699,1,1
1,17584.0,0.000000,0.0,9.4,60.785714,1,1
2,17789.0,0.000000,0.0,18.3,124.313772,1,1
3,17681.0,0.000000,0.0,4.4,27.883495,0,0
4,17515.0,-1.666667,-50.0,26.7,94.154381,1,1


In [23]:
test.head()

,first_redeem_date,express_points_spent_mean,express_points_spent_sum,regular_points_received_last_month,avg_spend_per_day
0,17684.0,0.0,0.0,8.5,35.767386
1,17410.0,0.0,0.0,7.2,49.829561
2,17725.0,0.0,0.0,6.6,60.716078
3,17754.0,0.0,0.0,47.5,150.277455
4,NaN,0.0,0.0,64.7,94.315789


In [24]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60012 entries, 0 to 60011
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   first_redeem_date                   58547 non-null  float64
 1   express_points_spent_mean           60012 non-null  float64
 2   express_points_spent_sum            60012 non-null  float64
 3   regular_points_received_last_month  60012 non-null  float64
 4   avg_spend_per_day                   60012 non-null  float64
dtypes: float64(5)
memory usage: 2.3 MB


In [25]:
test['first_redeem_date'].fillna(test['first_redeem_date'].median(), inplace=True)

In [26]:
test.head()

,first_redeem_date,express_points_spent_mean,express_points_spent_sum,regular_points_received_last_month,avg_spend_per_day
0,17684.0,0.0,0.0,8.5,35.767386
1,17410.0,0.0,0.0,7.2,49.829561
2,17725.0,0.0,0.0,6.6,60.716078
3,17754.0,0.0,0.0,47.5,150.277455
4,17684.0,0.0,0.0,64.7,94.315789


In [27]:
train.shape, test.shape

((140027, 7), (60012, 5))

In [29]:
params = {
    'max_depth':2,
    'learning_rate': 0.2, 
    'n_estimators': 100,
    'nthread':40,
    'n_gpus':0,
    'seed':42
}

estimator = XGBClassifier(
    **params
)

model = ClassTransformation(
    estimator=estimator
)

In [31]:
model.fit(
    X=train.drop(
            ['treatment_flg', 'purchased'], 
            axis=1
        ),
    y=train['purchased'],
    treatment=train['treatment_flg']
)

ClassTransformation(estimator=XGBClassifier(learning_rate=0.2, max_depth=2,
                                            n_gpus=0, nthread=40, seed=42))

In [32]:
tr_pred = model.predict(train.drop(['treatment_flg', 'purchased'], axis=1))

ts_pred = model.predict(test)

In [35]:
df_train_scores = train[['treatment_flg', 'purchased']]\
    .assign(uplift_score=tr_pred)
    
train_score = uplift_at_k(
    y_true=df_train_scores['purchased'],
    uplift=df_train_scores['uplift_score'],
    treatment=df_train_scores['treatment_flg'],
    strategy='overall'
)

print(f'Uplift@30% on train data: {train_score:.3f}')

Uplift@30% on train data: 0.115


In [36]:
test['pred'] = pd.Series(ts_pred)
test.head()

,first_redeem_date,express_points_spent_mean,express_points_spent_sum,regular_points_received_last_month,avg_spend_per_day,pred
0,17684.0,0.0,0.0,8.5,35.767386,0.021187
1,17410.0,0.0,0.0,7.2,49.829561,0.015494
2,17725.0,0.0,0.0,6.6,60.716078,0.026544
3,17754.0,0.0,0.0,47.5,150.277455,0.005215
4,17684.0,0.0,0.0,64.7,94.315789,0.017720


In [37]:
clients_id = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/x5_data/test.csv')
clients_id

,client_id
0,a9a604ed6e
1,ebd7360016
2,908cd9b8e8
3,dceb8ce861
4,f4f0ac6b06
...,...
60007,4762fb04c5
60008,533d1516e7
60009,c93b21a707
60010,3fe0668de5


In [38]:
clients_id['pred'] = pd.Series(ts_pred)
clients_id

,client_id,pred
0,a9a604ed6e,0.021187
1,ebd7360016,0.015494
2,908cd9b8e8,0.026544
3,dceb8ce861,0.005215
4,f4f0ac6b06,0.017720
...,...,...
60007,4762fb04c5,0.023039
60008,533d1516e7,0.014399
60009,c93b21a707,-0.070756
60010,3fe0668de5,0.065725


In [39]:
clients_id['pred'] = (clients_id['pred'] + 1)/2

In [41]:
clients_id.to_csv('answer.csv', index=False)